In [1]:
import pandas as pd

BASE_PATH = '/home/baldasseroniluisesteban/buckets/b1/datasets/'

df = pd.read_csv(BASE_PATH + 'competencia_02_fe.csv')
df.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,865361379,202101,1,0,0,34,134,445.55,5462.66,385.70,...,4.0,0.0,-779.25,0.0,4074.0,779.25,2.0,0.0,35.19,CONTINUA
1,865361379,202102,1,0,0,34,135,525.97,5734.45,380.13,...,-3.0,0.0,-788.60,0.0,4102.0,861.70,2.0,0.0,35.19,CONTINUA
2,865361379,202103,1,0,0,34,136,2075.33,7553.68,1994.59,...,1.0,0.0,-872.05,0.0,4133.0,988.35,2.0,0.0,46.92,CONTINUA
3,865361379,202104,1,0,0,34,137,-1077.29,6202.98,-1159.95,...,2.0,0.0,-1000.21,0.0,4163.0,988.35,2.0,0.0,46.92,CONTINUA
4,865361379,202105,1,0,0,34,138,541.40,5178.31,475.75,...,5.0,0.0,-1000.21,0.0,4194.0,988.35,2.0,0.0,46.92,CONTINUA


In [2]:
import pandas as pd

# Paso 1: Diccionario para valores 'CASO_NULL'
caso_null_dict = {
    "cdescubierto_preacordado": -1,
    "ccaja_seguridad": -1,
    "Master_madelantopesos": -1,
    "Visa_madelantopesos": -1,
    "Master_madelantodolares": -1,
    "Visa_madelantodolares": -1,
    "Master_mpagado": -1,
    "Visa_mpagado": -1,
    "mpagodeservicios": -1,
    "Master_cadelantosefectivo": -1,
    "Visa_cadelantosefectivo": -1,
    "Master_delinquency": -1,
    "Visa_delinquency": -1,
    "Master_status": -1,
    "Visa_status": -1,
    # Añade otras columnas con 'CASO_NULL' si es necesario
}

# Paso 2: Identificar columnas categóricas y numéricas
categorical_cols = [
    "cdescubierto_preacordado",
    "ccaja_seguridad",
    "active_quarter",
    "cliente_vip",
    "internet",
    "tcuentas",
    "ctarjeta_debito",
    "ctarjeta_visa",
    "ctarjeta_master",
    "tcallcenter",
    "thomebanking",
    "tmobile_app",
    "Master_delinquency",
    "Visa_delinquency",
    "Master_status",
    "Visa_status",
    # Añade otras columnas categóricas si corresponde
]

# Excluir columnas ya manejadas por 'caso_null_dict'
categorical_cols = [col for col in categorical_cols if col not in caso_null_dict]

# Identificar columnas numéricas restantes
numerical_cols = df.columns.difference(categorical_cols + list(caso_null_dict.keys()))

# Paso 3.1: Imputar columnas con valores 'CASO_NULL'
df.fillna(value=caso_null_dict, inplace=True)

# Paso 3.2: Imputar columnas categóricas restantes con la moda
for col in categorical_cols:
    if col in df.columns and df[col].isnull().sum() > 0:  # Verificar existencia de la columna
        mode = df[col].mode()
        if not mode.empty:  # Comprobar que la moda no esté vacía
            df[col].fillna(mode[0], inplace=True)
        else:
            df[col].fillna(0, inplace=True)

# Paso 3.3: Imputar columnas numéricas con la mediana
for col in numerical_cols:
    if col in df.columns and df[col].isnull().sum() > 0:  # Verificar existencia de la columna
        median = df[col].median()
        if not pd.isna(median):  # Verificar que la mediana no sea NaN
            df[col].fillna(median, inplace=True)

# Paso 4: Verificar que no queden valores faltantes
print("Valores faltantes por columna:")
print(df.isnull().sum())

print("\nTotal de valores faltantes en el DataFrame:", df.isnull().sum().sum())

# Paso 5: Convertir columnas categóricas a tipo 'category' (opcional)
for col in categorical_cols:
    if col in df.columns:  # Verificar existencia de la columna
        df[col] = df[col].astype('category')



/tmp/ipykernel_3253/2019023745.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode[0], inplace=True)
/tmp/ipykernel_3253/2019023745.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df

Valores faltantes por columna:
numero_de_cliente          0
foto_mes                   0
active_quarter             0
cliente_vip                0
internet                   0
                          ..
Visa_mconsumototal         0
Visa_cconsumos             0
Visa_cadelantosefectivo    0
Visa_mpagominimo           0
clase_ternaria             0
Length: 155, dtype: int64

Total de valores faltantes en el DataFrame: 0


In [ ]:
df.to_csv(BASE_PATH + 'competencia_02_sin_nulos.csv.gz', index=False, compression='gzip')

In [ ]:
df.shape

In [ ]:

def generar_lags(df, periodos=[1, 2, 3, 6, 12]):
    df_lags = df.copy()  # Hacer una copia del DataFrame original
    
    for col in df.columns:  # Iterar sobre todas las columnas del DataFrame
        for p in periodos:  # Generar los lags para los períodos definidos
            df_lags[f'{col}_lag_{p}'] = df[col].shift(p)  # Crear las columnas de lag
    
    return df_lags

# Ejemplo de uso:
df_lags = generar_lags(df_sin_nulos)